### Dynamic Programming
+ 分析子问题的重复性
+ 子问题进行存储
+ Solution 要进行解析

functools:
https://www.jianshu.com/p/178788297a5c

In [1]:
import time
from collections import defaultdict
from functools import wraps
from functools import lru_cache

In [9]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [10]:
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i + 1] = p #多少米对应的价格

In [11]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [13]:
# cal function time
def call_time(f, args):
    start = time.time()
    f(args)
    print('used time is {}'.format(time.time() - start))

In [15]:
def func_1(n):
    for i in range(n):
        print(i)

In [16]:
call_time(func_1, 10)

0
1
2
3
4
5
6
7
8
9
used time is 0.0010175704956054688


In [25]:
function_called_times = defaultdict(int)

def get_call_times(func):
    @wraps(func)
    def _inner(args):
        """it's inner function"""
        # global function_called_times
        function_called_times[func.__name__] += 1
        result = func(args)
        print("function called time is {}".format(function_called_times[func.__name__]))
        return result
    return _inner

In [19]:
func_1 = get_call_times(func_1) # 装饰器，得到新的func_1

In [27]:
func_1(10)

0
1
2
3
4
5
6
7
8
9
function called time is 2


In [28]:
@get_call_times
def func_1(n):
    """
    n: input parameter
    """
    for i in range(n):
        print(i)

In [29]:
help(func_1)

Help on function func_1 in module __main__:

func_1(n)
    n: input parameter



In [36]:
func_1(8)

0
1
2
3
4
5
6
7
function called time is 8


In [37]:
#测试下其他函数
@get_call_times
def func_slow(n):
    for i in range(n):
        time.sleep(0.2)
        print(i)

In [38]:
func_slow(6)

0
1
2
3
4
5
function called time is 1


$$ r_n = max(p_n, r_1 + r_{n-1}, r_2 + r_{n-2} ...... r_{n-1} + r_1) $$

In [61]:
solution = {}

In [62]:
#进行存储
def memo(func):
    cache = {} #缓存
    @wraps(func)
    def _wrap(n):
        if n in cache: result = cache[n] #n如果已经有了，直接返回结果
        else:
            result = func(n) #接着拆分
            cache[n] = result #把结果存进缓存中
        return result
    return _wrap

In [63]:
@memo
def r(n):
    max_price, max_split = max([(price[n], 0)] + [[r(i) + r(n - i), i]for i in range(1 ,n)], key=lambda x: x[0])
    solution[n] = (n - max_split, max_split)
    return max_price

In [64]:
r(38)

118

In [65]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3),
 21: (11, 10),
 22: (11, 11),
 23: (22, 1),
 24: (22, 2),
 25: (22, 3),
 26: (24, 2),
 27: (25, 2),
 28: (22, 6),
 29: (28, 1),
 30: (28, 2),
 31: (28, 3),
 32: (22, 10),
 33: (22, 11),
 34: (33, 1),
 35: (33, 2),
 36: (33, 3),
 37: (35, 2),
 38: (36, 2)}

In [152]:
#解析 solution
def parse_solution(n):
    left_split, right_split = solution[n]
    if right_split == 0: return [left_split]
    return parse_solution(left_split) + parse_solution(right_split)

In [153]:
parse_solution(38)

[11, 11, 11, 3, 2]

### Edit Distance

https://blog.csdn.net/chichoxian/article/details/53944188

编辑距离的作用主要是用来比较两个字符串的相似度的

编辑距离，又称Levenshtein距离（莱文斯坦距离也叫做Edit Distance），是指两个字串之间，由一个转成另一个所需的最少编辑操作次数，如果它们的距离越大，说明它们越是不同。许可的编辑操作包括将一个字符替换成另一个字符，插入一个字符，删除一个字符。

PS: 此部分代码完全参照老师的，会按照网上其他示例代码再做深一步的理解

In [8]:
solution = {}

In [11]:
'''
装饰器用一个有记忆的调用包装一个函数，它可以保存最近maxsize次调用。
当使用同样的参数定期调用费时或I/O绑定的函数时，它可以节省时间。
'''
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candicates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))
        
    candicates.append(both_forward)
    
    min_distance, operation = min(candicates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation
    
    return min_distance

In [4]:
edit_distance('ABCDE', 'ABCCEF')

2

In [5]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

In [12]:
edit_distance('12345', '54321')

4

In [13]:
solution

{('1', '5'): 'SUB 1 => 5',
 ('1', '54'): 'ADD 4',
 ('1', '543'): 'ADD 3',
 ('1', '5432'): 'ADD 2',
 ('1', '54321'): '',
 ('12', '5'): 'DEL 2',
 ('12', '54'): 'SUB 2 => 4',
 ('12', '543'): 'ADD 3',
 ('12', '5432'): '',
 ('12', '54321'): 'ADD 1',
 ('123', '5'): 'DEL 3',
 ('123', '54'): 'DEL 3',
 ('123', '543'): '',
 ('123', '5432'): 'ADD 2',
 ('123', '54321'): 'ADD 1',
 ('1234', '5'): 'DEL 4',
 ('1234', '54'): '',
 ('1234', '543'): 'DEL 4',
 ('1234', '5432'): 'SUB 4 => 2',
 ('1234', '54321'): 'ADD 1',
 ('12345', '5'): '',
 ('12345', '54'): 'DEL 5',
 ('12345', '543'): 'DEL 5',
 ('12345', '5432'): 'DEL 5',
 ('12345', '54321'): 'SUB 5 => 1'}